In [50]:
import pandas as pd
X_full = pd.read_csv('data_msk.csv')
X_full = X_full.replace(to_replace='empty', value=None)
X_full.head()

,region,address,price,total_area,kitchen_area,living_area,rooms_count,floor,floors_number,build_date,...,cargo_elevator,metro,metro_distance,metro_transport,district,is_apartments,is_auction,repair,with_furniture,windows_type
0,msk,"Москва, Ильинка",196080000,88.5,30.5,34.2,2,2,5,None,...,None,"Площадь Революции,Театральная,Китай-город","3,6,6","walk,walk,walk","Тверской,ЦАО",0,0,None,None,None
1,msk,"Россия, Москва, Московский международный делов...",491101853,294.6,None,None,4,80,85,None,...,1,"Деловой центр,Международная,Деловой центр","3,3,4","walk,walk,walk","Пресненский,ЦАО",1,1,None,None,None
2,msk,"Москва, Большой Толмачёвский переулок",156750000,85.6,15.5,34.6,2,2,14,None,...,1,"Полянка,Боровицкая,Третьяковская","2,20,4","transport,walk,transport","Якиманка,ЦАО",0,0,None,None,None
3,msk,"Москва, М. Бронная ул., 32",99000000,110.5,15.0,61.0,3,2,6,1912,...,0,"Маяковская,Пушкинская,Тверская","10,10,10","walk,walk,walk","Пресненский,ЦАО",None,1,None,None,None
4,msk,"Москва, Дубининская улица, 59к6",32551037,58.0,None,24.1,2,3,27,None,...,None,"Серпуховская,Павелецкая,Добрынинская","18,20,4","walk,walk,transport","Даниловский,ЮАО",0,0,None,None,None


In [52]:
X_full['kitchen_area'] = X_full['kitchen_area'].astype('float')
X_full['living_area'] = X_full['living_area'].astype('float')
X_full['build_date'] = X_full['build_date'].astype('float')
X_full['isСomplete'] = X_full['isСomplete'].astype('float')
X_full['complitation_year'] = X_full['complitation_year'].astype('float')
X_full['passenger_elevator'] = X_full['passenger_elevator'].astype('float')
X_full['cargo_elevator'] = X_full['cargo_elevator'].astype('float')
X_full['is_apartments'] = X_full['is_apartments'].astype('float')

In [53]:
X_full = X_full[X_full.price < X_full.price.quantile(q=0.95)]

In [54]:
X_train = X_full.drop('price', axis=1)

In [55]:
Y = X_full['price']

In [56]:
X_train['rooms_count'] = X_train['rooms_count'].fillna(1.0)

In [57]:
X_train['kitchen_area'] = X_train['kitchen_area'].fillna(X_train['kitchen_area'].mean())

In [58]:
X_train['living_area'] = X_train['living_area'].fillna(X_train['living_area'].mean())

In [59]:
X_train.isna().sum()

region                   0
address                  0
total_area               0
kitchen_area             0
living_area              0
rooms_count              0
floor                    0
floors_number            0
build_date             790
isСomplete             931
complitation_year      910
house_material         209
parking                554
decoration            1086
balcony                968
longitude                0
latitude                 0
passenger_elevator     383
cargo_elevator         710
metro                   33
metro_distance          33
metro_transport         33
district                 0
is_apartments          222
is_auction               0
repair                1389
with_furniture        1389
windows_type          1389
dtype: int64

In [60]:
X_train = X_train.drop(columns={'address', 'district'}, axis=1)

In [61]:
X_train['build_date'] = X_train['build_date'].fillna(2025)

In [62]:
X_train['complitation_year'] = X_train['complitation_year'].fillna(2023)

In [63]:
X_train.loc[X_train['build_date'].astype(int) < 2024, 'isСomplete'] = 1
# Если дата окончания строительства (complitation_year) > 2024, то дом не построен
X_train.loc[X_train['complitation_year'].astype(int) > 2024, 'isСomplete'] = 0

print('После:', X_train['isСomplete'].value_counts())

После: isСomplete
1.0    695
0.0    361
Name: count, dtype: int64


In [64]:
X_train.loc[X_train['build_date'].astype(int) >= X_train['complitation_year'].astype(int), 'isСomplete'] = 0

In [65]:
most_value_count = X_train['house_material'].value_counts().index[0]
most_value_count

'monolith'

In [66]:
X_train['house_material'] = X_train['house_material'].fillna(most_value_count)

In [67]:
X_train = X_train.drop(columns={'metro', 'metro_distance', 'metro_transport','decoration', 'balcony', 'parking'}, axis=1)

In [68]:
X_train.loc[X_train['floors_number'].astype(int) > 5, 'passenger_elevator'] = 1
X_train.loc[X_train['floors_number'].astype(int) <= 5, 'passenger_elevator'] = 0

In [69]:
X_train.loc[X_train['floors_number'].astype(int) > 5, 'cargo_elevator'] = 1
X_train.loc[X_train['floors_number'].astype(int) <= 5, 'cargo_elevator'] = 0

In [70]:
X_train['is_apartments'] = X_train['is_apartments'].fillna(0.0)

In [71]:
X_train

,region,total_area,kitchen_area,living_area,rooms_count,floor,floors_number,build_date,isСomplete,complitation_year,house_material,longitude,latitude,passenger_elevator,cargo_elevator,is_apartments,is_auction,repair,with_furniture,windows_type
0,msk,88.5,30.500000,34.2,2,2,5,2025.0,0.0,2024.0,monolith,37.623509,55.755201,0.0,0.0,0.0,0,None,None,None
2,msk,85.6,15.500000,34.6,2,2,14,2025.0,0.0,2024.0,monolith,37.619913,55.739474,1.0,1.0,0.0,0,None,None,None
3,msk,110.5,15.000000,61.0,3,2,6,1912.0,1.0,2023.0,brick,37.593524,55.764197,1.0,1.0,0.0,1,None,None,None
4,msk,58.0,16.357027,24.1,2,3,27,2025.0,0.0,2024.0,monolith,37.635652,55.717862,1.0,1.0,0.0,0,None,None,None
5,msk,153.0,14.000000,102.0,4,6,7,1914.0,1.0,2023.0,brick,37.601439,55.743330,1.0,1.0,0.0,1,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,msk,77.8,14.000000,44.3,3,7,17,2025.0,0.0,2023.0,panel,37.356953,55.515558,1.0,1.0,0.0,1,None,None,None
1459,msk,59.2,6.000000,44.6,3,7,9,1970.0,1.0,2023.0,panel,37.490865,55.776137,1.0,1.0,0.0,1,None,None,None
1460,msk,48.6,8.000000,32.0,2,2,5,1953.0,1.0,2023.0,brick,37.484891,55.793288,0.0,0.0,0.0,1,None,None,None
1461,msk,45.0,6.000000,30.0,2,8,9,1973.0,1.0,2023.0,monolith,37.647190,55.681899,1.0,1.0,0.0,1,None,None,None


In [72]:
X_train.isna().sum()

region                   0
total_area               0
kitchen_area             0
living_area              0
rooms_count              0
floor                    0
floors_number            0
build_date               0
isСomplete               0
complitation_year        0
house_material           0
longitude                0
latitude                 0
passenger_elevator       0
cargo_elevator           0
is_apartments            0
is_auction               0
repair                1389
with_furniture        1389
windows_type          1389
dtype: int64

In [73]:
X_train = X_train.drop(columns={'repair', 'with_furniture', 'windows_type'}, axis=1)

In [76]:
X_train = X_train.drop(columns='region')

In [77]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1389 entries, 0 to 1462
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   total_area          1389 non-null   float64
 1   kitchen_area        1389 non-null   float64
 2   living_area         1389 non-null   float64
 3   rooms_count         1389 non-null   int64  
 4   floor               1389 non-null   int64  
 5   floors_number       1389 non-null   int64  
 6   build_date          1389 non-null   float64
 7   isСomplete          1389 non-null   float64
 8   complitation_year   1389 non-null   float64
 9   house_material      1389 non-null   object 
 10  longitude           1389 non-null   float64
 11  latitude            1389 non-null   float64
 12  passenger_elevator  1389 non-null   float64
 13  cargo_elevator      1389 non-null   float64
 14  is_apartments       1389 non-null   float64
 15  is_auction          1389 non-null   int64  
dtypes: float64(

In [78]:
from sklearn.model_selection import train_test_split

X_trained, X_val, y_trained, y_val = train_test_split(X_train, Y, test_size=0.2, random_state=42)

In [79]:
import numpy as np
from catboost import CatBoostRegressor, Pool
categorical_features = ['house_material']
train_pool = Pool(X_trained, y_trained, categorical_features)
test_pool = Pool(X_val, y_val, categorical_features)


model = CatBoostRegressor(iterations=500, max_depth=5, learning_rate=0.05, random_seed=1066, logging_level='Silent')
model.fit(X_trained, y_trained, eval_set=test_pool, cat_features=categorical_features, use_best_model=True, early_stopping_rounds=10)

print(f"Model is fitted: {str(model.is_fitted())}")
print(f"Model params: {model.get_params()}")

Model is fitted: True
Model params: {'iterations': 500, 'learning_rate': 0.05, 'loss_function': 'RMSE', 'random_seed': 1066, 'logging_level': 'Silent', 'max_depth': 5}


In [81]:
model.score(X_trained, y_trained)

0.8877109478696585

In [83]:
model.save_model('model_msk.cbm')

In [99]:
from catboost import CatBoostRegressor
loaded_model = CatBoostRegressor()
loaded_model.load_model('model_msk.cbm')


In [ ]:
loaded_model.predict([95.0, ])

In [86]:
X_val.columns

Index(['total_area', 'kitchen_area', 'living_area', 'rooms_count', 'floor',
       'floors_number', 'build_date', 'isСomplete', 'complitation_year',
       'house_material', 'longitude', 'latitude', 'passenger_elevator',
       'cargo_elevator', 'is_apartments', 'is_auction'],
      dtype='object')

In [ ]:
cols = ['total_area', 'kitchen_area', 'living_area', 'rooms_count', 'floor',
       'floors_number', 'build_date', 'isСomplete', 'complitation_year',
       'house_material', 'longitude', 'latitude', 'passenger_elevator',
       'cargo_elevator', 'is_apartments', 'is_auction']
cat_cols = ['house_material']

for_predict = Pool()

In [88]:
model.predict(X_val.iloc[0])

90361034.16440007

In [90]:
y_val

580     100000000
174      15471700
246      40651000
699      55000000
278      11781000
          ...    
160      20661010
318      28359680
1136     28000000
433      97000000
737      28046440
Name: price, Length: 278, dtype: int64

In [93]:
print(X_val.iloc[0].values)

[95.0 25.0 52.21097708082026 3 11 21 2025.0 0.0 2023.0 'monolith'
 37.555391 55.762809 1.0 1.0 0.0 1]


In [98]:
s = "['total_area', 'kitchen_area', 'living_area', 'rooms_count', 'floor', 'floors_number', 'build_date', 'isСomplete', 'complitation_year', 'house_material', 'longitude', 'latitude', 'passenger_elevator','cargo_elevator', 'is_apartments', 'is_auction']"
s = s.replace("'", "")
s

'[total_area, kitchen_area, living_area, rooms_count, floor, floors_number, build_date, isСomplete, complitation_year, house_material, longitude, latitude, passenger_elevator,cargo_elevator, is_apartments, is_auction]'

In [96]:
model.predict([95.0, 25.0, 52.21097708082026, 3, 11, 21, 2025.0, 0.0, 2023.0, 'monolith', 37.555391, 55.762809, 1.0, 1.0, 0.0, 1])
38157178
90361034

90361034.16440007

In [101]:
loaded_model.predict([95.0, 25.0, 52.21097708082026, 3, 11, 21, 2025.0, 0.0, 2023.0, 'monolith', 37.555391, 55.762809, 1.0, 1.0, 0.0, 1])

92212618.2313072